In [10]:
#Required Libraries 
%pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.2.24/llama_cpp_python-0.2.24-cp311-cp311-win_amd64.whl
import sys
!{sys.executable} -m pip install huggingface_hub

!{sys.executable} -m pip install llama-cpp-python

  Using cached https://github.com/abetlen/llama-cpp-python/releases/download/v0.2.24/llama_cpp_python-0.2.24-cp311-cp311-win_amd64.whl (1.8 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Downloading Model 
## Huggingface Authentication 
from huggingface_hub import login
from dotenv import load_dotenv
import os
load_dotenv()

login(token=os.environ.get("HUGGINGFACE_TOKEN"))


In [6]:
## Actual download of model 
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="QuantFactory/Meta-Llama-3-8B-Instruct-GGUF",
    filename="Meta-Llama-3-8B-Instruct.Q4_K_M.gguf"
)

print(f"Model downloaded to: {model_path}")

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/79/f2/79f21025e377180e4ec0e3968bca4612bb9c99fa84e70cb7815186c42a858124/86c8ea6c8b755687d0b723176fcd0b2411ef80533d23e2a5030f845d13ab2db7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Meta-Llama-3-8B-Instruct.Q4_K_M.gguf%3B+filename%3D%22Meta-Llama-3-8B-Instruct.Q4_K_M.gguf%22%3B&Expires=1745793276&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTc5MzI3Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzc5L2YyLzc5ZjIxMDI1ZTM3NzE4MGU0ZWMwZTM5NjhiY2E0NjEyYmI5Yzk5ZmE4NGU3MGNiNzgxNTE4NmM0MmE4NTgxMjQvODZjOGVhNmM4Yjc1NTY4N2QwYjcyMzE3NmZjZDBiMjQxMWVmODA1MzNkMjNlMmE1MDMwZjg0NWQxM2FiMmRiNz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=ukilDzLn1lmrNHIMjz3Akp9gEHijexEfT59t2%7EHZq10TGczme4f28LSnW8noQudGUkzg4whdxVhynQsBfvHO21fePhoNT-k0CRd8m%7EZ33dvVc2Yce23ejmRGuhxJAqHYemklu1pqTGZaPakDXvoBkKD1NoqYP84aj6eIe%7E4YxkKk0LS-OainYrRDKhTbCBce7mKmCdQm9C0BlMIvsKWIGCzW

Model downloaded to: C:\Users\lgrei\.cache\huggingface\hub\models--QuantFactory--Meta-Llama-3-8B-Instruct-GGUF\snapshots\86e0c07efa3f1b6f06ea13e31b1e930dce865ae4\Meta-Llama-3-8B-Instruct.Q4_K_M.gguf


In [11]:
## Loading model into memory 
from llama_cpp import Llama

llama3_mtg = Llama(
    model_path=model_path,
    n_ctx=2048,
    n_threads=4,
    use_mlock=False
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [12]:
## Example Prompt 
prompt = """Generate Magic: The Gathering card rules text with the following features:

- Type: Creature
- Subtypes: Elf, Warrior
- Color Identity: Green
- Mana Cost: {1}{G}
- Power/Toughness: 2/3
- Keywords: Trample, Reach

Rules Text:"""

output = llama3_mtg(prompt, max_tokens=100, stop=["\n\n"])
print(output["choices"][0]["text"].strip())

Whenever ~ deals combat damage to a player, that player discards a card.


In [14]:
# Card Generation Creation 

def generate_card(
    type: list,
    subtypes: list,
    color_identity: list,
    mana_cost: str,
    power: str,
    toughness: str,
    keywords: list,
    max_tokens: int = 250
):
    
    prompt = f"""Create a Magic: The Gathering card using the following characteristics:

- Type: {', '.join(type)}
- Subtypes: {', '.join(subtypes)}
- Color Identity: {', '.join(color_identity)}
- Mana Cost: {mana_cost}
- Power/Toughness: {power}/{toughness}
- Keywords: {', '.join(keywords)}

Return the card in the following format:

Name: <The card's name>,
Rules Text: <Describes the card’s abilities, effects, and how it functions during gameplay. Use formal Magic: The Gathering rules text. Include triggers, costs, and any conditions needed.>
"""

    result = llama3_mtg(prompt, max_tokens=max_tokens)
    text = result["choices"][0]["text"].strip()

    # Ensure it starts with Rules Text for consistency
    text = f"Rules Text: {text}"

    # Parse model response
    card = {}
    for line in text.splitlines():
        if ':' in line:
            key, value = line.split(':', 1)
            card[key.strip()] = value.strip()

    # Fallbacks
    fallback_name = next((line for line in text.splitlines() if line.strip()), "Unnamed")
    card_name = card.get("Name", fallback_name)
    final_rules = card.get("Rules Text", "[No rules text generated]")

    # Display
    card_display = f"""\
Name: {card_name}
Type: {type}
Subtypes: {', '.join(subtypes)}
Color Identity: {', '.join(color_identity)}
Mana Cost: {mana_cost}
Power/Toughness: {power}/{toughness}
Keywords: {', '.join(keywords)}
Rules Text:
{final_rules}
"""

    return {
        "card_display": card_display,
        "name": card_name,
        "rules_text": final_rules
    }



In [ ]:
## Example Prompt 
card = generate_card(
    type="Creature",
    subtypes=["Zombie", "Cleric"],
    color_identity=["Black"],
    mana_cost="{2}{B}",
    power="2",
    toughness="2",
    keywords=["Deathtouch", "Lifelink"]
)

print(card["card_display"])

TypeError: generate_card() got an unexpected keyword argument 'type_'